In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [11]:
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": "C:\\Users\\gualas\\Desktop\\prueba",
})

In [12]:
driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',chrome_options=chrome_options)
# Acceder a la página
driver.get("https://ourworldindata.org/world-population-growth")

time.sleep(5)
# Cerrar ventana emergente de las cookies
try:
    boton_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/div/div[2]/button')))
    boton_cookies.click()
except:
    pass

# Hacer clic en el botón "Download chart"
enlace_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-track-note="chart-click-download"]')))
enlace_descarga.click()


# Esperar a que aparezca el botón de descarga real
boton_descarga = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/article/div[2]/div[2]/div/div/section[1]/figure/div/div[3]/div/div[4]/div/div[2]/div/button')))
boton_descarga.click()


C:\Users\gualas\AppData\Local\Temp\ipykernel_11084\3154625403.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',chrome_options=chrome_options)
C:\Users\gualas\AppData\Local\Temp\ipykernel_11084\3154625403.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r'C:\Users\gualas\Desktop\ChromeDriver\chromedriver.exe',chrome_options=chrome_options)
